In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import unicodedata as un

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.
pd.set_option('display.max_colwidth', -1)

In [ ]:
train_path = '../input/train.csv'
test_path = '../input/test.csv'
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df = train_df.drop(['qid'], axis=1)
test_df = test_df.drop(['qid'], axis=1)

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
# Drop the column which is not necessary
train_df = train_df.drop(['qid'], axis=1)
test_df = test_df.drop(['qid'], axis=1)
train_df.head()

In [ ]:
train_df.sample()

In [ ]:
# Count the number of rows with each label
train_df.groupby('target').count()

In [ ]:
# To see what insincere question looks like
train_df.where(train_df.target == 1).dropna().sample(10)

In [ ]:
# To see what insincere question looks like
train_df.where(train_df.target == 0).dropna().sample(10)

In [ ]:
# train_df['question_text'] = train_df['question_text'].str.replace('?','').str.replace('"','').str.lower()

In [ ]:
train_df.sample(10)

In [ ]:
train_df['question_text'] = train_df['question_text'].str.lower()

In [ ]:
train_df.sample(20)

In [ ]:
table = dict.fromkeys(i for i in range(sys.maxunicode) 
                        if not un.category(chr(i)).startswith(('Ll'))
                        and (i != 32 or i >= 122))

In [ ]:
train_df['question_text'] = train_df['question_text'].str.translate(table)

In [ ]:
train_df.sample(10)